In [13]:
import os
import glob
import shutil
import json
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook as tqdm

In [14]:
data_dir = '../data/data_aug/paste/'
out_dir = data_dir
images_path = os.path.join(data_dir, 'images')
annot_path = data_dir + os.path.join(data_dir, 'labels')
json_name = 'coco_format_%s.json'

In [15]:
# separating images by video folders
images = glob.glob(images_path+'/*.png')
for image in tqdm(images):
    root = image.split('images/')[0]
    video = image.split('images/')[-1][:17]
    if not os.path.isdir(os.path.join(root, video)):
        os.makedirs(os.path.join(root, video))
    shutil.copyfile(image, os.path.join(root, video, image[-17:]))    

In [9]:
def get_img_size(datapath):
    img_pil = Image.open(datapath)
    width, height = img_pil.size
    return width, height


def get_groundtruth(img_path, annotation_folder):
    annot_path = None
    if annotation_folder is not None:
        # look for annotation file
        annot_path = _find_annot_file(img_path, annotation_folder)

    if annot_path is not None:
        frame_number = _get_frame_number(img_path)
        annotation = AnnotationImage(frame_number, annot_path)
        boxes, labels = annotation.get_bboxes_labels()

        return boxes, labels

    return [], []

In [22]:
sets = [
        'Train',
        'Test',
]


categories = [
    '__background__',
    'tire',
    ]

img_id = 0
ann_id = 0
cat_id = 1
category_dict = {
    #'__background__': 0,
    'tire': 0,
    }

for data_set in tqdm(sets):
    print(f'Starting {data_set}')
    
    ann_dict = {}
    images = []
    annotations = []
    
#     filenames = glob.glob(os.path.join(data_dir, data_set,'*.png'))
    
    for (dirpath, dirnames, filenames) in tqdm(os.walk(os.path.join(data_dir, data_set))):
        if len(filenames) == 0:
            continue
    
        for filename in tqdm(filenames):
            if filename.lower().endswith(('.png')):               
                image = {}
                
                img_name_short = os.path.join(os.path.split(dirpath)[-1], filename)
                img_name = os.path.join(dirpath, filename)
                    
                width = get_img_size(img_name)[0]
                height = get_img_size(img_name)[1]

                image['file_name'] = img_name_short
                image['width'] = width
                image['height'] = height
                image['id'] = img_id
                
                #read .txt in images folder
                anno_filepath = os.path.join(data_dir, 'images', os.path.split(dirpath)[-1] + 
                                       '_'+os.path.split(filename)[-1].replace('.png','.txt'))
                
                if os.path.isfile(anno_filepath):
                
                    with open(anno_filepath, 'r') as marks:

                        for mark in marks.readlines():
                            rect = mark.replace('\n','').split('\t')
                            objid,rcx,rcy,rw,rh = map(float,rect)
                            x0 = int((rcx-rw/2)*width)
                            y0 = int((rcy-rh/2)*height)
                            w = int(rw*width)
                            h = int(rh*height)

                            ann = {}
                            ann['id'] = ann_id
                            ann['image_id'] = img_id
                            ann['category_id'] = 0
                            ann['iscrowd'] = 0
                            ann['area'] = w*h
                            ann['bbox'] = (x0,y0,w,h)

                            annotations.append(ann)
                            ann_id += 1
                
                #read .txt in labels folder                
                else:
                    anno_filepath = os.path.join(data_dir, 'labels', os.path.split(dirpath)[-1] + 
                                       '_'+os.path.split(filename)[-1].replace('.png','.txt'))

                    with open(anno_filepath, 'r') as marks:

                        for mark in marks.readlines():
                            rect = mark.replace('\n','').split(' ')
                            objid,rcx,rcy,rw,rh = map(float,rect)
                            x0 = int((rcx-rw/2)*width)
                            y0 = int((rcy-rh/2)*height)
                            w = int(rw*width)
                            h = int(rh*height)

                            ann = {}
                            ann['id'] = ann_id
                            ann['image_id'] = img_id
                            ann['category_id'] = 0
                            ann['iscrowd'] = 0
                            ann['area'] = w*h
                            ann['bbox'] = (x0,y0,w,h)

                            annotations.append(ann)
                            ann_id += 1
                        
                        
                images.append(image)
                img_id += 1

            
    ann_dict['images'] = images
    categories = [{"id": category_dict[name], "name": name} for name in category_dict]
    ann_dict['categories'] = categories
    ann_dict['annotations'] = annotations
    print("Num categories: %s" % len(categories))
    print("Num images: %s" % len(images))
    print("Num annotations: %s" % len(annotations))
    
    with open(os.path.join(out_dir, json_name % data_set), 'w') as outfile:
        outfile.write(json.dumps(ann_dict))

Starting Train


Num categories: 1
Num images: 4950
Num annotations: 5419
Starting Test


Num categories: 1
Num images: 5715
Num annotations: 11210
